# Complete Guide to Ollama Python API

This notebook provides a comprehensive, step-by-step guide to using the Ollama API with Python. We'll cover everything from installation to advanced usage patterns.

## Table of Contents
1. [Prerequisites and Installation](#prerequisites)
2. [Basic Setup and Configuration](#setup)
3. [Simple Chat Interactions](#basic-chat)
4. [Working with Different Models](#models)
5. [Streaming Responses](#streaming)
6. [Advanced Features](#advanced)
7. [Error Handling and Best Practices](#best-practices)
8. [Real-world Examples](#examples)

## 1. Prerequisites and Installation {#prerequisites}

Before we start, make sure you have:
- Python 3.7 or higher
- Ollama installed and running on your system
- At least one model downloaded in Ollama

### Installing Required Packages

In [ ]:
# Install the official Ollama Python library
!pip install ollama

# Optional: Install additional helpful libraries
!pip install requests json-stream

### Verify Ollama is Running

Let's check if Ollama is running and accessible:

In [ ]:
import requests
import json

try:
    response = requests.get('http://localhost:11434/api/tags')
    if response.status_code == 200:
        models = response.json()
        print("✅ Ollama is running!")
        print(f"Available models: {len(models.get('models', []))}")
        for model in models.get('models', []):
            print(f"  - {model['name']}")
    else:
        print(f"❌ Ollama responded with status code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to Ollama. Make sure it's running on localhost:11434")
    print("Run 'ollama serve' in your terminal to start Ollama")

If you see the message `✅ Ollama is running!`, you're all set to start using the Ollama Python API.

## 2. Basic Setup and Configuration {#setup}

Now let's import the Ollama library and set up our basic configuration:

In [ ]:
import ollama
from typing import Dict, List, Any
import time

# Create an Ollama client
client = ollama.Client()

# You can also specify a custom host if Ollama is running elsewhere
# client = ollama.Client(host='http://localhost:11434')

print("✅ Ollama client initialized successfully!")

### List Available Models

Let's see what models we have available:

In [ ]:
def list_models():
    """List all available models in Ollama"""
    try:
        models = client.list()
        print(f"📋 Available Models ({len(models['models'])}):\n")
        
        for model in models['models']:
            name = model['name']
            size = model.get('size', 0) / (1024**3)  # Convert to GB
            modified = model.get('modified_at', 'Unknown')
            print(f"  🤖 {name}")
            print(f"     Size: {size:.1f} GB")
            print(f"     Modified: {modified}\n")
            
        return [model['name'] for model in models['models']]
    except Exception as e:
        print(f"❌ Error listing models: {e}")
        return []

available_models = list_models()

## 3. Simple Chat Interactions {#basic-chat}

Let's start with basic chat functionality. We'll create a simple function to chat with a model:

In [ ]:
def simple_chat(model_name: str, message: str) -> str:
    """Send a simple message to a model and get a response"""
    try:
        response = client.chat(
            model=model_name,
            messages=[
                {
                    'role': 'user',
                    'content': message,
                },
            ],
        )
        return response['message']['content']
    except Exception as e:
        return f"Error: {e}"

# Example usage
if available_models:
    model = available_models[0]  # Use the first available model
    print(f"🤖 Using model: {model}\n")
    
    question = "What is Python and why is it popular?"
    print(f"👤 User: {question}\n")
    
    answer = simple_chat(model, question)
    print(f"🤖 Assistant: {answer}")
else:
    print("❌ No models available. Please download a model first using 'ollama pull <model_name>'")

### Interactive Chat Function

Let's create a more interactive chat function that maintains conversation history:

In [ ]:
class OllamaChat:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.conversation_history = []
        self.client = ollama.Client()
    
    def send_message(self, message: str) -> str:
        """Send a message and maintain conversation history"""
        # Add user message to history
        self.conversation_history.append({
            'role': 'user',
            'content': message
        })
        
        try:
            # Get response from model
            response = self.client.chat(
                model=self.model_name,
                messages=self.conversation_history
            )
            
            assistant_message = response['message']['content']
            
            # Add assistant response to history
            self.conversation_history.append({
                'role': 'assistant',
                'content': assistant_message
            })
            
            return assistant_message
            
        except Exception as e:
            return f"Error: {e}"
    
    def clear_history(self):
        """Clear the conversation history"""
        self.conversation_history = []
        print("🗑️ Conversation history cleared")
    
    def get_history(self):
        """Get the current conversation history"""
        return self.conversation_history
    
    def print_history(self):
        """Print the conversation history in a readable format"""
        print("📜 Conversation History:\n")
        for i, message in enumerate(self.conversation_history, 1):
            role_emoji = "👤" if message['role'] == 'user' else "🤖"
            role_name = "User" if message['role'] == 'user' else "Assistant"
            print(f"{i}. {role_emoji} {role_name}: {message['content']}\n")

# Example usage
if available_models:
    chat = OllamaChat(available_models[0])
    print(f"💬 Started chat with {available_models[0]}\n")
    
    # Have a conversation
    response1 = chat.send_message("Hello! What's your name?")
    print(f"🤖: {response1}\n")
    
    response2 = chat.send_message("Can you remember what I just asked you?")
    print(f"🤖: {response2}\n")
    
    # Show conversation history
    chat.print_history()

## 4. Working with Different Models {#models}

Let's explore how to work with different models and compare their responses:

In [ ]:
def compare_models(models: List[str], question: str):
    """Compare responses from different models"""
    print(f"❓ Question: {question}\n")
    print("=" * 80)
    
    for model in models:
        print(f"\n🤖 Model: {model}")
        print("-" * 40)
        
        start_time = time.time()
        response = simple_chat(model, question)
        end_time = time.time()
        
        print(f"Response: {response}")
        print(f"⏱️ Time taken: {end_time - start_time:.2f} seconds")
        print("=" * 80)

# Example: Compare different models (if you have multiple)
if len(available_models) > 1:
    compare_models(
        available_models[:2],  # Compare first two models
        "Explain quantum computing in simple terms"
    )
else:
    print("ℹ️ Only one model available. Download more models to compare responses.")
    print("Example: ollama pull llama2 && ollama pull codellama")

### Model Information and Management

Let's create functions to get detailed information about models:

In [ ]:
def get_model_info(model_name: str):
    """Get detailed information about a specific model"""
    try:
        info = client.show(model_name)
        
        print(f"📊 Model Information: {model_name}\n")
        print(f"License: {info.get('license', 'Unknown')}")
        print(f"Template: {info.get('template', 'Unknown')}")
        print(f"Parameters: {info.get('parameters', 'Unknown')}")
        
        if 'modelfile' in info:
            print(f"\n📄 Modelfile:\n{info['modelfile']}")
            
        return info
    except Exception as e:
        print(f"❌ Error getting model info: {e}")
        return None

# Get info for the first available model
if available_models:
    model_info = get_model_info(available_models[0])

## 5. Streaming Responses {#streaming}

For longer responses, streaming can provide a better user experience. Let's implement streaming:

In [ ]:
def stream_chat(model_name: str, message: str):
    """Stream a chat response token by token"""
    print(f"🤖 {model_name} (streaming): ", end="", flush=True)
    
    try:
        stream = client.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': message}],
            stream=True,
        )
        
        full_response = ""
        for chunk in stream:
            content = chunk['message']['content']
            print(content, end="", flush=True)
            full_response += content
        
        print("\n")  # New line after streaming is complete
        return full_response
        
    except Exception as e:
        print(f"\n❌ Streaming error: {e}")
        return ""

# Example streaming
if available_models:
    print("📡 Streaming Example:\n")
    stream_chat(
        available_models[0], 
        "Write a short story about a robot learning to paint"
    )

### Advanced Streaming with Progress Tracking

Let's create a more sophisticated streaming function with progress tracking:

In [ ]:
import sys
from datetime import datetime

class StreamingChat:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.client = ollama.Client()
    
    def stream_with_stats(self, message: str, show_stats: bool = True):
        """Stream response with statistics"""
        start_time = time.time()
        token_count = 0
        full_response = ""
        
        if show_stats:
            print(f"🚀 Starting stream at {datetime.now().strftime('%H:%M:%S')}")
            print(f"📝 Prompt: {message}\n")
            print(f"🤖 {self.model_name}: ", end="", flush=True)
        
        try:
            stream = self.client.chat(
                model=self.model_name,
                messages=[{'role': 'user', 'content': message}],
                stream=True,
            )
            
            for chunk in stream:
                content = chunk['message']['content']
                print(content, end="", flush=True)
                full_response += content
                token_count += len(content.split())
            
            end_time = time.time()
            duration = end_time - start_time
            
            if show_stats:
                print(f"\n\n📊 Statistics:")
                print(f"   ⏱️ Duration: {duration:.2f} seconds")
                print(f"   🔤 Approximate tokens: {token_count}")
                print(f"   ⚡ Tokens/second: {token_count/duration:.1f}")
                print(f"   📏 Response length: {len(full_response)} characters")
            
            return {
                'response': full_response,
                'duration': duration,
                'token_count': token_count,
                'tokens_per_second': token_count/duration if duration > 0 else 0
            }
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            return None

# Example usage
if available_models:
    streaming_chat = StreamingChat(available_models[0])
    result = streaming_chat.stream_with_stats(
        "Explain the concept of machine learning in detail"
    )

## 6. Advanced Features {#advanced}

Let's explore some advanced features of the Ollama API:

### Custom System Prompts

System prompts help define the behavior and personality of the model:

In [ ]:
def chat_with_system_prompt(model_name: str, system_prompt: str, user_message: str):
    """Chat with a custom system prompt"""
    try:
        response = client.chat(
            model=model_name,
            messages=[
                {
                    'role': 'system',
                    'content': system_prompt
                },
                {
                    'role': 'user',
                    'content': user_message
                }
            ]
        )
        return response['message']['content']
    except Exception as e:
        return f"Error: {e}"

# Example: Create a coding assistant
if available_models:
    system_prompt = """
    You are a helpful Python programming assistant. You should:
    - Provide clear, well-commented code examples
    - Explain complex concepts in simple terms
    - Always include error handling where appropriate
    - Suggest best practices and optimizations
    """
    
    print("🔧 System Prompt Example - Python Assistant:\n")
    response = chat_with_system_prompt(
        available_models[0],
        system_prompt,
        "How do I read a CSV file and handle missing values?"
    )
    print(f"🤖: {response}\n")
    
    # Example: Create a creative writing assistant
    creative_prompt = """
    You are a creative writing assistant. You should:
    - Write in an engaging, descriptive style
    - Use vivid imagery and metaphors
    - Create compelling characters and scenarios
    - Always end with a cliffhanger or thought-provoking question
    """
    
    print("✨ System Prompt Example - Creative Writer:\n")
    response = chat_with_system_prompt(
        available_models[0],
        creative_prompt,
        "Write a short paragraph about a mysterious door"
    )
    print(f"🤖: {response}")

### Temperature and Generation Parameters

Control the creativity and randomness of responses:

In [ ]:
def chat_with_options(model_name: str, message: str, temperature: float = 0.7, 
                     top_p: float = 0.9, top_k: int = 40):
    """Chat with custom generation parameters"""
    try:
        response = client.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': message}],
            options={
                'temperature': temperature,
                'top_p': top_p,
                'top_k': top_k,
            }
        )
        return response['message']['content']
    except Exception as e:
        return f"Error: {e}"

# Compare different temperature settings
if available_models:
    question = "Write a creative opening line for a science fiction story"
    temperatures = [0.1, 0.7, 1.2]
    
    print(f"🌡️ Temperature Comparison for: '{question}'\n")
    
    for temp in temperatures:
        print(f"Temperature {temp} ({
            'Conservative' if temp < 0.5 else 
            'Balanced' if temp < 1.0 else 'Creative'
        }):")
        response = chat_with_options(available_models[0], question, temperature=temp)
        print(f"🤖: {response}\n")

### Working with Embeddings

Generate embeddings for text similarity and semantic search:

In [ ]:
import numpy as np
from typing import List

def get_embeddings(model_name: str, texts: List[str]):
    """Get embeddings for a list of texts"""
    embeddings = []
    
    for text in texts:
        try:
            response = client.embeddings(
                model=model_name,
                prompt=text
            )
            embeddings.append(response['embedding'])
        except Exception as e:
            print(f"❌ Error getting embedding for '{text}': {e}")
            embeddings.append(None)
    
    return embeddings

def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors"""
    if vec1 is None or vec2 is None:
        return 0.0
    
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    return dot_product / (norm1 * norm2)

# Example: Text similarity
if available_models:
    # Note: Not all models support embeddings. This will work with models like nomic-embed-text
    texts = [
        "The cat sat on the mat",
        "A feline rested on the rug",
        "Dogs are loyal animals",
        "Python is a programming language"
    ]
    
    print("🔍 Embedding Example (Note: Requires embedding-capable model):\n")
    
    # Try with the first available model (may not support embeddings)
    try:
        embeddings = get_embeddings(available_models[0], texts)
        
        if all(emb is not None for emb in embeddings):
            print("📊 Text Similarity Matrix:\n")
            
            for i, text1 in enumerate(texts):
                for j, text2 in enumerate(texts):
                    if i <= j:  # Only show upper triangle
                        similarity = cosine_similarity(embeddings[i], embeddings[j])
                        print(f"{i+1}-{j+1}: {similarity:.3f} | '{text1}' <-> '{text2}'")
        else:
            print("⚠️ This model doesn't support embeddings. Try 'ollama pull nomic-embed-text'")
    except Exception as e:
        print(f"⚠️ Embeddings not supported by this model: {e}")
        print("Try downloading an embedding model: ollama pull nomic-embed-text")

## 7. Error Handling and Best Practices {#best-practices}

Let's implement robust error handling and best practices:

In [ ]:
import logging
from functools import wraps
import time

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def retry_on_failure(max_retries: int = 3, delay: float = 1.0):
    """Decorator to retry function calls on failure"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    logger.warning(f"Attempt {attempt + 1} failed: {e}")
                    
                    if attempt < max_retries - 1:
                        time.sleep(delay * (2 ** attempt))  # Exponential backoff
            
            raise last_exception
        return wrapper
    return decorator

class RobustOllamaClient:
    def __init__(self, host: str = 'http://localhost:11434'):
        self.client = ollama.Client(host=host)
        self.host = host
    
    def is_server_available(self) -> bool:
        """Check if Ollama server is available"""
        try:
            self.client.list()
            return True
        except Exception:
            return False
    
    def is_model_available(self, model_name: str) -> bool:
        """Check if a specific model is available"""
        try:
            models = self.client.list()
            return any(model['name'] == model_name for model in models['models'])
        except Exception:
            return False
    
    @retry_on_failure(max_retries=3)
    def safe_chat(self, model_name: str, messages: List[Dict], **kwargs) -> Dict:
        """Safe chat with error handling and retries"""
        # Validate inputs
        if not self.is_server_available():
            raise ConnectionError(f"Ollama server not available at {self.host}")
        
        if not self.is_model_available(model_name):
            raise ValueError(f"Model '{model_name}' not available")
        
        if not messages:
            raise ValueError("Messages cannot be empty")
        
        # Make the request
        response = self.client.chat(
            model=model_name,
            messages=messages,
            **kwargs
        )
        
        return response
    
    def chat_with_timeout(self, model_name: str, message: str, timeout: int = 30) -> str:
        """Chat with timeout handling"""
        import signal
        
        def timeout_handler(signum, frame):
            raise TimeoutError(f"Request timed out after {timeout} seconds")
        
        # Set up timeout
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)
        
        try:
            response = self.safe_chat(
                model_name,
                [{'role': 'user', 'content': message}]
            )
            return response['message']['content']
        finally:
            signal.alarm(0)  # Cancel the alarm

# Example usage
robust_client = RobustOllamaClient()

print("🛡️ Robust Client Examples:\n")

# Test server availability
if robust_client.is_server_available():
    print("✅ Server is available")
    
    if available_models:
        model = available_models[0]
        
        # Test model availability
        if robust_client.is_model_available(model):
            print(f"✅ Model '{model}' is available")
            
            # Safe chat example
            try:
                response = robust_client.safe_chat(
                    model,
                    [{'role': 'user', 'content': 'Hello, how are you?'}]
                )
                print(f"🤖: {response['message']['content']}")
            except Exception as e:
                print(f"❌ Safe chat failed: {e}")
        else:
            print(f"❌ Model '{model}' is not available")
else:
    print("❌ Server is not available")

### Input Validation and Sanitization

Always validate and sanitize user inputs:

In [ ]:
import re
from typing import Optional

class InputValidator:
    @staticmethod
    def validate_message(message: str, max_length: int = 10000) -> str:
        """Validate and sanitize user message"""
        if not isinstance(message, str):
            raise TypeError("Message must be a string")
        
        if not message.strip():
            raise ValueError("Message cannot be empty")
        
        if len(message) > max_length:
            raise ValueError(f"Message too long. Maximum {max_length} characters allowed")
        
        # Remove potentially harmful characters
        sanitized = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', message)
        
        return sanitized.strip()
    
    @staticmethod
    def validate_model_name(model_name: str) -> str:
        """Validate model name"""
        if not isinstance(model_name, str):
            raise TypeError("Model name must be a string")
        
        if not model_name.strip():
            raise ValueError("Model name cannot be empty")
        
        # Basic model name validation (alphanumeric, hyphens, underscores, colons)
        if not re.match(r'^[a-zA-Z0-9_:-]+$', model_name):
            raise ValueError("Invalid model name format")
        
        return model_name.strip()
    
    @staticmethod
    def validate_temperature(temperature: float) -> float:
        """Validate temperature parameter"""
        if not isinstance(temperature, (int, float)):
            raise TypeError("Temperature must be a number")
        
        if temperature < 0 or temperature > 2:
            raise ValueError("Temperature must be between 0 and 2")
        
        return float(temperature)

def safe_chat_with_validation(model_name: str, message: str, 
                             temperature: float = 0.7) -> Optional[str]:
    """Chat function with comprehensive input validation"""
    try:
        # Validate inputs
        validated_model = InputValidator.validate_model_name(model_name)
        validated_message = InputValidator.validate_message(message)
        validated_temp = InputValidator.validate_temperature(temperature)
        
        # Use robust client
        response = robust_client.safe_chat(
            validated_model,
            [{'role': 'user', 'content': validated_message}],
            options={'temperature': validated_temp}
        )
        
        return response['message']['content']
        
    except (TypeError, ValueError) as e:
        logger.error(f"Validation error: {e}")
        return f"Validation error: {e}"
    except Exception as e:
        logger.error(f"Chat error: {e}")
        return f"Chat error: {e}"

# Example usage
if available_models:
    print("🔒 Validation Examples:\n")
    
    # Valid input
    response = safe_chat_with_validation(
        available_models[0],
        "What is the capital of France?",
        0.5
    )
    print(f"✅ Valid input: {response[:100]}...\n")
    
    # Invalid temperature
    response = safe_chat_with_validation(
        available_models[0],
        "Hello",
        5.0  # Invalid temperature
    )
    print(f"❌ Invalid temperature: {response}\n")
    
    # Empty message
    response = safe_chat_with_validation(
        available_models[0],
        "",  # Empty message
        0.7
    )
    print(f"❌ Empty message: {response}")

## 8. Real-world Examples {#examples}

Let's create some practical, real-world examples:

### Example 1: Document Summarizer

In [ ]:
class DocumentSummarizer:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.client = ollama.Client()
    
    def summarize(self, text: str, summary_type: str = "brief") -> str:
        """Summarize a document"""
        prompts = {
            "brief": "Provide a brief 2-3 sentence summary of the following text:",
            "detailed": "Provide a detailed summary with key points of the following text:",
            "bullet": "Summarize the following text as bullet points:"
        }
        
        prompt = prompts.get(summary_type, prompts["brief"])
        full_prompt = f"{prompt}\n\n{text}"
        
        try:
            response = self.client.chat(
                model=self.model_name,
                messages=[{'role': 'user', 'content': full_prompt}]
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}"
    
    def extract_key_points(self, text: str, num_points: int = 5) -> str:
        """Extract key points from text"""
        prompt = f"Extract the {num_points} most important key points from the following text:\n\n{text}"
        
        try:
            response = self.client.chat(
                model=self.model_name,
                messages=[{'role': 'user', 'content': prompt}]
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}"

# Example usage
if available_models:
    summarizer = DocumentSummarizer(available_models[0])
    
    sample_text = """
    Artificial Intelligence (AI) has become one of the most transformative technologies of the 21st century. 
    It encompasses various subfields including machine learning, natural language processing, computer vision, 
    and robotics. Machine learning, in particular, has seen remarkable progress with the development of deep 
    learning algorithms that can process vast amounts of data and identify complex patterns. These advances 
    have led to breakthrough applications in healthcare, where AI can assist in medical diagnosis and drug 
    discovery, in transportation with autonomous vehicles, and in finance for fraud detection and algorithmic 
    trading. However, the rapid advancement of AI also raises important ethical considerations, including 
    concerns about job displacement, privacy, bias in algorithms, and the need for responsible AI development. 
    As we move forward, it's crucial to balance innovation with ethical considerations to ensure that AI 
    benefits society as a whole.
    """
    
    print("📄 Document Summarizer Example:\n")
    print("Original text length:", len(sample_text), "characters\n")
    
    # Brief summary
    brief = summarizer.summarize(sample_text, "brief")
    print(f"📝 Brief Summary:\n{brief}\n")
    
    # Key points
    key_points = summarizer.extract_key_points(sample_text, 3)
    print(f"🔑 Key Points:\n{key_points}")

### Example 2: Code Assistant

In [ ]:
class CodeAssistant:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.client = ollama.Client()
    
    def explain_code(self, code: str, language: str = "python") -> str:
        """Explain what a piece of code does"""
        prompt = f"Explain what this {language} code does, line by line:\n\n```{language}\n{code}\n```"
        
        try:
            response = self.client.chat(
                model=self.model_name,
                messages=[{'role': 'user', 'content': prompt}]
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}"
    
    def debug_code(self, code: str, error_message: str, language: str = "python") -> str:
        """Help debug code with error message"""
        prompt = f"""
        I'm getting this error with my {language} code:
        
        Error: {error_message}
        
        Code:
        ```{language}
        {code}
        ```
        
        Please help me identify the issue and suggest a fix.
        """
        
        try:
            response = self.client.chat(
                model=self.model_name,
                messages=[{'role': 'user', 'content': prompt}]
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}"
    
    def optimize_code(self, code: str, language: str = "python") -> str:
        """Suggest optimizations for code"""
        prompt = f"""
        Please review this {language} code and suggest optimizations for better performance, 
        readability, or best practices:
        
        ```{language}
        {code}
        ```
        """
        
        try:
            response = self.client.chat(
                model=self.model_name,
                messages=[{'role': 'user', 'content': prompt}]
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}"
    
    def generate_tests(self, code: str, language: str = "python") -> str:
        """Generate unit tests for code"""
        prompt = f"""
        Generate comprehensive unit tests for this {language} code:
        
        ```{language}
        {code}
        ```
        
        Include edge cases and error handling tests.
        """
        
        try:
            response = self.client.chat(
                model=self.model_name,
                messages=[{'role': 'user', 'content': prompt}]
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}"

# Example usage
if available_models:
    code_assistant = CodeAssistant(available_models[0])
    
    sample_code = """
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

result = fibonacci(10)
print(result)
    """
    
    print("💻 Code Assistant Example:\n")
    
    # Explain code
    explanation = code_assistant.explain_code(sample_code)
    print(f"📖 Code Explanation:\n{explanation}\n")
    
    # Optimize code
    optimization = code_assistant.optimize_code(sample_code)
    print(f"⚡ Optimization Suggestions:\n{optimization}")

### Example 3: Multi-turn Conversation Manager

In [ ]:
import json
from datetime import datetime
from pathlib import Path

class ConversationManager:
    def __init__(self, model_name: str, save_conversations: bool = True):
        self.model_name = model_name
        self.client = ollama.Client()
        self.conversations = {}
        self.save_conversations = save_conversations
        self.save_dir = Path("conversations")
        
        if save_conversations:
            self.save_dir.mkdir(exist_ok=True)
    
    def start_conversation(self, conversation_id: str, system_prompt: str = None) -> str:
        """Start a new conversation"""
        messages = []
        if system_prompt:
            messages.append({
                'role': 'system',
                'content': system_prompt
            })
        
        self.conversations[conversation_id] = {
            'messages': messages,
            'created_at': datetime.now().isoformat(),
            'model': self.model_name
        }
        
        return f"Started conversation '{conversation_id}' with {self.model_name}"
    
    def send_message(self, conversation_id: str, message: str) -> str:
        """Send a message in a conversation"""
        if conversation_id not in self.conversations:
            return f"Conversation '{conversation_id}' not found. Start it first."
        
        # Add user message
        self.conversations[conversation_id]['messages'].append({
            'role': 'user',
            'content': message,
            'timestamp': datetime.now().isoformat()
        })
        
        try:
            # Get response
            response = self.client.chat(
                model=self.model_name,
                messages=[
                    {k: v for k, v in msg.items() if k in ['role', 'content']}
                    for msg in self.conversations[conversation_id]['messages']
                ]
            )
            
            assistant_message = response['message']['content']
            
            # Add assistant response
            self.conversations[conversation_id]['messages'].append({
                'role': 'assistant',
                'content': assistant_message,
                'timestamp': datetime.now().isoformat()
            })
            
            # Save conversation
            if self.save_conversations:
                self._save_conversation(conversation_id)
            
            return assistant_message
            
        except Exception as e:
            return f"Error: {e}"
    
    def get_conversation_summary(self, conversation_id: str) -> str:
        """Get a summary of the conversation"""
        if conversation_id not in self.conversations:
            return f"Conversation '{conversation_id}' not found."
        
        conv = self.conversations[conversation_id]
        message_count = len([msg for msg in conv['messages'] if msg['role'] in ['user', 'assistant']])
        
        return f"""
Conversation: {conversation_id}
Model: {conv['model']}
Created: {conv['created_at']}
Messages: {message_count}
        """.strip()
    
    def _save_conversation(self, conversation_id: str):
        """Save conversation to file"""
        filename = self.save_dir / f"{conversation_id}.json"
        with open(filename, 'w') as f:
            json.dump(self.conversations[conversation_id], f, indent=2)
    
    def load_conversation(self, conversation_id: str) -> str:
        """Load conversation from file"""
        filename = self.save_dir / f"{conversation_id}.json"
        
        if not filename.exists():
            return f"Conversation file '{filename}' not found."
        
        try:
            with open(filename, 'r') as f:
                self.conversations[conversation_id] = json.load(f)
            return f"Loaded conversation '{conversation_id}'"
        except Exception as e:
            return f"Error loading conversation: {e}"
    
    def list_conversations(self) -> List[str]:
        """List all active conversations"""
        return list(self.conversations.keys())

# Example usage
if available_models:
    conv_manager = ConversationManager(available_models[0])
    
    print("💬 Conversation Manager Example:\n")
    
    # Start a conversation with a system prompt
    system_prompt = "You are a helpful Python programming tutor. Always provide clear explanations and examples."
    print(conv_manager.start_conversation("python_tutorial", system_prompt))
    
    # Have a conversation
    response1 = conv_manager.send_message("python_tutorial", "What are Python decorators?")
    print(f"\n🤖: {response1[:200]}...\n")
    
    response2 = conv_manager.send_message("python_tutorial", "Can you show me a simple example?")
    print(f"🤖: {response2[:200]}...\n")
    
    # Get conversation summary
    summary = conv_manager.get_conversation_summary("python_tutorial")
    print(f"📊 Summary:\n{summary}")
    
    # List conversations
    conversations = conv_manager.list_conversations()
    print(f"\n📋 Active conversations: {conversations}")

## Conclusion

This notebook has covered comprehensive usage of the Ollama Python API, including:

- ✅ Basic setup and configuration
- ✅ Simple and advanced chat interactions
- ✅ Working with different models
- ✅ Streaming responses
- ✅ Advanced features (system prompts, parameters, embeddings)
- ✅ Error handling and best practices
- ✅ Real-world examples (document summarizer, code assistant, conversation manager)

### Next Steps

1. **Experiment** with different models and parameters
2. **Build** your own applications using these patterns
3. **Explore** additional Ollama features and models
4. **Implement** proper logging and monitoring for production use
5. **Consider** rate limiting and resource management for high-volume applications

### Useful Resources

- [Ollama Documentation](https://ollama.ai/)
- [Ollama Python Library](https://github.com/ollama/ollama-python)
- [Available Models](https://ollama.ai/library)
- [Model Cards and Documentation](https://ollama.ai/library)

Happy coding with Ollama! 🚀